# Dynamic Programming

We hope dynamic programming isn't a completely new idea to you.  If it is, not to worry, this part of the assignment is designed to make sure everyone has the base they need to do well in the rest of the class.

## Purpose

The simplest way to think about dynamic programming, "DP", is nothing more than as an approach to optimize an algorithm:

1.  Find an algorithm that "brute forces" a problem
2.  Notice that it fits into the the class of algorithms DP can optimize
3.  Implement algorithm using DP

Dynamic programming does **not** have anything to do with on-the-fly code generation, online reinforcement learning, editing/optimizing compiled code while it's running... or anything else fancy.  So what does it do?  Read on!

## Approach

It boils down to a simple idea:  _don't let your code do the same work twice_.  In practice, this means that you should cache intermediate results (say, in a table).  Each time you need to evaluate a function with the same input after the first, you can look up the result in the cache rather than recomputing it.

You will sometimes hear DP is used for problems whose solutions are combinations of the solutions to "optimal subproblems".  Unpacking this a little bit:  if each solution is just a combination of smaller versions of the same problem, and you cache the results of the smaller problems as you go, to the extent that you need those solutions in more than one place, you'll get a speed up.  For many problems, an exponential time naive implementation can be made to run in polynomial time.

**Don't let this sound complicated to you.**  All we're doing is adding a cache to avoid recomputing an "expensive" function.  Any time you see a problem that sounds like, "Try every possible combination of X and pick the best", you should think "I wonder if there are any partial results here we can cache and reuse".

## Why is this important for natural language processing?

In the second half of the course, we'll study tasks where the desired output is a sequence, a tree, or another type of rich structure. Unlike simple classification problems, we can't easily enumerate all the possible outputs - because there are exponentially many!

In many such tasks, we can still decompose them into sub-problems, like predicting a single branch of a tree. Then by using dynamic programming, we can devise an efficient algorithm to find the highest-scoring solution. We'll make heavy use of this approach when we study parsing and sequence labeling tasks like part-of-speech tagging.

Dynamic programming is also useful as a conceptual tool. Decomposing a large problem into subproblems can be as useful for us as for computers, since it lets us reason about small, controlled cases and extend them into a more complex solution. Dynamic programming is also closely related to recursion - a concept underlying many fundamental ideas in both computer science and linguistics.

## Example: Climbing the stairs

Let's say you want to compute the total number of ways you can run up a flight of stairs.  At each step $k$ you can either:

- Take a regular step up to $k + 1$
- Take a big step up to $k + 2$.

Starting on stair $0$ (the bottom floor), how many unique ways are there to get up a staircase with $n$ stairs?

If you've not encountered this kind of problem before, **spend a minute pondering** how you might compute this.  It's common to feel "lost" in the combinatorial explosion of options.

### Hint

Could you figure out the number of ways up a staircase of height $n$ if you were told how many ways you might get up staircases of height $n - 1$ and $n - 2$?  That is, if you knew the solution to a _subproblem_, could you use that to compute the solution to your real problem?

### Further Hint

Say you can get to the second-to-the-top stair in 8 ways (and then take a big step) and the next-to-the-top stair in 13 ways (and then take a small step).  How many ways to the top?  8 + 13 = 21.  (Note that you don't have to count "get to second-to-the-top and take two regular steps" separately because the "13" already includes them.)

### Solution

In general, if there are $ways(n - 1)$ to get to the $n - 1$ stair, you can use any of those methods to get there and then take a regular step to get to stair $n$.  Similarly, if you have $ways(n - 2)$ to get to the $n - 2$ step and then take a big step to stair $n$.  There is no other way to get to stair $n$ except by one of those two sets of options.  There is no overlap between those sets of sequences as sequences in the first set always end in a "regular" step and those sequences in the second set always end in a "big" step.

In math, $ways(n) = ways(n-1) + ways(n-2)$.

Also notice (imagine, or draw a picture), $ways(1) = 1$ and $ways(2) = 2$.

*Aside: you may recognize this as the famous [Fibonacci series](https://en.wikipedia.org/wiki/Fibonacci_number).*

The next cell shows one implementation.

In [1]:
def naive_ways(n):
    """Return the ways up a staircase of length n. Uses a naive algorithm."""
    if n == 1:
        return 1
    if n == 2:
        return 2
    
    return naive_ways(n - 1) + naive_ways(n - 2)

naive_ways(3)

**Great!**

Unfortunately, this implementation gets very slow for large $n$. The cell below will print some timing information.

In [1]:
# TIMING INFO FOR PART A
for n in range(20, 25+1):
    print("n=%d: " % n),
    %timeit -n25 naive_ways(n)

In [2]:
# Try with slightly-larger n.
%timeit -n1 naive_ways(35)

If we try to handle a staircase with 100 steps, it'll take a _very_ long time with our naive implementation.  (If you decide to try it, you'll want to interrupt your kernel instead of wait for the result!)

## A. Short Answer Questions

Give *brief* answers to the following in the cell below.

1. Based on the timing numbers from the `# TIMING INFO FOR PART A` cell above, approximately how much slower is $ways(n)$ than $ways(n-1)$? (i.e. what is $\frac{time(ways(n))}{time(ways(n-1))}$, roughly?)

2.  Why is this so slow?  What calculations do we compute repeatedly?  Hint: consider the following diagram.
![DP diagram](dp.png)

3. **Food for thought (not graded):** Assuming that $time(ways(n)) = time(ways(n-1)) + time(ways(n-2))$, what is $\lim_{n\to\infty} \frac{time(ways(n))}{time(ways(n-1))}$?

## Apply Dynamic Programming

Let's see if we can compute the results more cleverly, by keeping a cache (or table) of intermediate results that we can re-use.

First answer the following questions, then use what you gleaned to examine the code.

### B. Short Answer Questions
1. What are the values of A, B, and C in this table?
<html><table>
<tr><td>n</td><td>ways(n)</td></tr>
<tr><td>1</td><td>1</td></tr>
<tr><td>2</td><td>2</td></tr>
<tr><td>3</td><td>3</td></tr>
<tr><td>4</td><td>5</td></tr>
<tr><td>5</td><td>8</td></tr>
<tr><td>6</td><td>13</td></tr>
<tr><td>7</td><td>$A$</td></tr>
<tr><td>8</td><td>$B$</td></tr>
<tr><td>9</td><td>$C$</td></tr>
</table>
</html>
<p>
2. To compute these values, did you look at $n=4$ or earlier?
<p>
3. What is the minimum number of values you need to keep as you fill the table from top to bottom, while maintaining the DP property of not recomputing any values?  
<p>

In [3]:
def dynamic_programming_ways(n):
    """Return the ways up a staircase of length n."""
    #  Initialize the cache to the answer for staircases of length one (1 way) and two (2 ways).
    cache = [1, 2]
    
    # Extend the cache, reusing the results we already have in the cache.
    # (For the mathematically inclined, you may see some
    #  parallels between DP and proof by induction.)
    for i in range(n - 2):
        cache.append(cache[i] + cache[i + 1])
        
    # We've extended the cache far enough that we can look up staircase of length n.
    # (Note that all the indices are one lower than the staircase, right from the
    #  time of initialization, hence the -1.)
    return cache[n - 1]

assert dynamic_programming_ways(1) == 1
assert dynamic_programming_ways(2) == 2
assert dynamic_programming_ways(100) == 573147844013817084101

%time dynamic_programming_ways(100)

We can tweak things a little bit to reduce the memory footprint.  As seen in the short answer questions and in the code, you only need `cache[i-2]` and `cache[i-1]` to compute `cache[i]`.  Keeping earlier results from the cache are a bit of a waste.

In [4]:
def dynamic_programming_ways_efficient(n):
    """Return the ways up a staircase of length n."""
    #  Initialize the cache to the answer for staircases of length one (1 way) and two (2 ways).
    #  (We need to do some special casing here for n=1.  We could avoid this by initializing
    #   cache_i_1 == 1 as well and changing the for loop to be range(n-1).  Think about why...)
    if n == 1:
        return 1
    
    cache_i_2 = 1
    cache_i_1 = 2
    
    # Extend the cache, reusing the results we already have in the cache.
    # (For the mathematically inclined, you'll be excused if you see some
    #  parallels between DP and proof by induction.)
    for i in range(n - 2):
        new_value = cache_i_1 + cache_i_2
        cache_i_2 = cache_i_1
        cache_i_1 = new_value
        
    # We've extended the cache far enough that we can look up staircase of length n.
    # (Note that all the indices are one lower than the staircase, right from the
    #  time of initialization, hence the -1.)
    return cache_i_1

assert dynamic_programming_ways_efficient(1) == 1
assert dynamic_programming_ways_efficient(2) == 2
assert dynamic_programming_ways_efficient(100) == 573147844013817084101

%time dynamic_programming_ways_efficient(100)

## Pipe cutting

Another problem suceptible to optimization with DP is pipe cutting.  Here, we are given a metal pipe of length $n$ and a price list, the price at which you can sell various parts of the pipe.  For the purposes of this exercise, let's write the code as though different parts of the pipe could yield different amounts of money, even if they're the same length.  Our objective is to cut the pipe into pieces such that they fetch the most revenue.

Similar to the stairs, we're going to slowly build a table up mapping the length of bars to the profit to be had.  If I have a bar of length 1, what's the most profitable way of cutting it up?  What about length 2?  Length 3? etc.  By keeping these intermediate results around, when we try cutting a length 10 pipe in one place, we can efficiently determine the profit made by optimally cutting the resulting left and right subpieces up.

Inspect the code below.

### Why do we care?

It turns out the pipe cutting problem is (very) closely related to DP algorithms for segmentation in NLP, as we'll see in more detail below. In Assignment 5, you'll implement the CKY algorithm for parsing, which is a sort of two-sided version of pipe-cutting that allows us to produce tree structures.

In [5]:
price_list = [0.0, -3.4, -5.7, 17.6, 2.2, 86.3]
def pipe_profit(left, right):
    """$ profit for the part of the bar in interval [left, right).
   
       Negative prices are a result of being unable to sell them
       and having to pay for disposal.  (Actually, as we'll learn
       later in the class, language models often produce -'ve scores,
       so we'll want to make sure our pipe splitting algorithm is robust
       to such scores when we use it in combination with a language model
       below.  Don't worry, you don't need to know anything about
       language models for this assignment.)
    """
    n = right - left
    if n < 0 or n >= len(price_list):
        return 0.0
    return price_list[n]

# DP cache for the function below.
def best_cuts(n, score):
    """Determine the optimal revenue possible by (optionally) cutting a bar
       of length n.

    Args:
      n: the length of the pipe
      score: a function that accepts "left" and "right and gives you the score
             (revenue) received for the segment of pipe extending the interval
             from [left, right)
    
    Returns:
      The best profit to be had for a pipe of length n.
    """
        
    profit_on_left_most = []
    for k in range(n + 1):
        # What if we don't split the k-length bar at all?
        profit_on_left_most.append(score(0, k))
        
        # Maybe split it up.  Pick a split location at "j", optimally split the left
        # hand side but keep the right hand side whole.
        # (Answer below why this doesn't skip any options.)
        for j in range(k):
            potential_profit = profit_on_left_most[j] + score(j, k)
            if potential_profit > profit_on_left_most[k]:
                profit_on_left_most[k] = potential_profit
    
    return profit_on_left_most[n]
   
profit_on_9 = round(best_cuts(9, pipe_profit), 1)
assert 100.5 == profit_on_9

profit_on_9

## C. Short Answer Questions

1. In the inner loop, why do we only need to try each location of a single cut (at `j`)?  Why don't we need to try cutting the bar into three or more pieces?

2. Answer the question posed in the code:  Why do we not need to optimally split both sides of `j`?  In other words, why can we get away with only taking the optimal split (`profit_on_left_most`) on the left and just `score()` the right?

3. Where are the cuts you should make for n=9?  Be sure to explain your work.  _(Hint:  Add a "print" to the code to see where you end up updating the profit made.  Leave this code in when you submit.)_

## Bookkeeping

Great!  We know we can make over a hundred dollars cutting up our length 9 bar!

Unfortunately, while we computed the revenue available by cutting the bar, we didn't actually track the cuts we need to make in order to earn it!

## D. Coding Exercise
**Finish modifying the code (in the cell below)** to keep track of the cuts you made in order to achieve the optimal revenue.

i.e. `potential_profit[4]` might be `(39.5, [1, 2])`.

In [6]:
# DP cache for the function below.
def best_cuts_with_trace(n, score):
    """Determine the optimal revenue possible by (optionally) cutting a bar
       of length n.

    Args:
      n: the length of the pipe
      score: a function that accepts "left" and "right and gives you the score
             (revenue) received for the segment of pipe extending the interval
             from [left, right)
    
    Returns:
      The best profit to be had for a pipe of length n.
    """
        
    profit_on_left_most = []
    for k in range(n + 1):
        # What if we don't split the k-length bar at all?
        profit_on_left_most.append((score(0, k), []))  # We did this part for you:  no cuts, so [].
        
        # Maybe split it up.  Pick a split location at "j", optimally split the left
        # hand side but keep the right hand side whole.
        # (Answer below why this doesn't skip any options.)
        for j in range(k):
            pass  # Placeholder so python doesn't complain about having no body.
            # YOUR CODE HERE
            # In the previous cell, this block looked like this:
            # potential_profit = profit_on_left_most[j] + score(j, k)
            # if potential_profit > profit_on_left_most[k]:
            #     profit_on_left_most[k] = potential_profit
            #
            # Hint:  When adding "j" to the list of cuts already required, use:
            # original_list + [j], not original_list.append(j).  You don't want to edit that original
            # list (since then it'll be wrong in its original place).
            
            
            # END YOUR CODE HERE

    return profit_on_left_most[n]

   
result = best_cuts_with_trace(9, pipe_profit)
# You may end up with [4, 1] (or if you edit a lot more code than you need to, other equivalent sets of cuts).
# At the very least, make sure you return them sorted as the code below relies on it.
# You should NOT end up with a cut at 0 or a cut at 9.  That's the end of the bar already - no need to cut again!
assert result == (100.5, [1, 4])
assert result[1] == sorted(result[1])
result

We'll see bookkeeping like this throughout the rest of the course.  For example, we'll want to know the optimal way to tag words in a sentence with their parts of speech.  The optimization will be over some likelihood of a particular assignment (rather than revenue).  This optimization is only useful to us however if we have a way to know what sequence of part of speech tags gave us that score, so we'll have to do this same kind of bookkeeping.

## Pipe cutting is segmentation
The next cell implements a very light wrapper around your best_cuts_with_trace function (it just takes the cut indexes your function returns and turns them into text to pretty-print).  It also implements a simple unigram language model (much simpler than what you will build in later assignments!). 

Don't worry about the details here, this is just a fun coda to the assignment!  Figuring out these details will be the work of the rest of the course.

Feel free to experiment with some sentences.  See if you can find at least one that breaks it, yielding a sub-optimal segmentation. If the dynamic programming algorithm is exact (finds the highest scoring split), how can it produce a bad solution?

In [8]:
import numpy as np

##
# Compute unigram counts from a simple corpus.
unigram_counts = {}
total_counts = 0
for line in open('english_uni_simplified_sorted_top').readlines():
    word_and_count = line.split()
    word = word_and_count[0].strip('"')
    count = int(word_and_count[1])
    unigram_counts[word] = count
    total_counts += count

def unigram_scoring_function(text, left, right):
    word = text[left:right]
    if word in unigram_counts:
        # Log probabilities, so we can add scores instead of multiplying
        return np.log(unigram_counts[word]) - np.log(total_counts)
    else:  
        # "Smoothing", encouraging in-vocabulary, or at least short OOV words.
        # We give a lower score to longer out-of-vocabulary spans.
        return -100 * (right - left)
        
##
# Use the pipe-cutting algorithm to segment text.
def segment(text):
    # We create a scoring lambda that accepts two parameters, "left" and "right", as required by the
    # code you implemented above.  However, we also need access to the "text" in order
    # to score the unigram.  A lambda captures the local variable "text" for this purpose.
    score_func = lambda left, right: unigram_scoring_function(text, left, right)
    
    # Call your function to slice the string.
    score, cuts = best_cuts_with_trace(len(text), score_func)
    
    # Imply a "cut" at the start and end of the text so that the list comprehension below is convenient.
    cuts = [0] + cuts + [len(text)]
    
    # Convert the list of cuts into a list of words.
    return score, [text[cuts[i] : cuts[i + 1]] for i in range(len(cuts) - 1)]

In [7]:
segment('helloworldhowareyou')

In [8]:
segment('downbythebay')

In [9]:
segment('wikipediaisareallystrongresourceontheinternet')

## Congratulations!

You're done with Dynamic Programming.

There is a completely optional section that shouldn't take very long, if you're keen to learn about edit distance.  We won't delve into much detail about this anywhere else in the class.

## (Optional) String edit distance

Another classic DP problem in the NLP space - but not one we otherwise will talk about in the course is the idea of "[edit distance](https://en.wikipedia.org/wiki/Levenshtein_distance)".  It's a way of measuring how many "edits" to one string you need to make in order to turn it into another.

We've provided two implementations below for you to play with.

1.  **levenshtein_cache:** The "cache everything in a dict" approach is first.  The keys are coordinates into a table that is len(str1) x len(str2) in size.

2.  **levenshtein_explicit:** Similar to the version of ways(n) that only keeps the previous two values at hand, the explicit ordering approach only keeps the immediately previous row of the table while building the next. Setting the verbose flag to this version prints each row of the table out as it computes it.

## E. (Optional) Short Answer Questions:

Give brief answers to the following in the cell below.

1. Let `n = len(str1)` and `m = len(str2)`. In terms of `n` and `m`, what is the size of the DP table (cache) for computing Levenshtein distance? _Hint: how many valid keys are there? Do we use all of them?_
<p>
2. Based on your answer to 1., what is the running time (in Big-O notation) of the edit distance algorithm? _Hint: it takes $O(1)$ work at each step, assuming we have the needed cache entries._
<p>

3. Consider transpositions (that is, swapping adjacent characters), such as `xy` -> `yx`. How can we compose a transposition from insertions, deletions, and substitutions? What is the edit distance between `wxyz` and `wyxz`?
<p>

4. Suppose we wanted to handle transpositions directly, rather than allowing our algorithm to compose them from other operations. (This might be useful if we want to score them differently.) If we have for the other operations:
```python
_ed(i - 1, j) + 1  # insertion
_ed(i, j - 1) + 1  # deletion
_ed(i - 1, j - 1) + substitution  # substitution, free if letters match  
```
what line would we add (calling `_ed`) to handle a transposition? (You may want to define a variable `transposition_match` to check that a transposition makes sense at the current position.) Based on your answer to 1. and 2., does this change the Big-O runtime of the algorithm?

In [7]:
def levenshtein_cache(str1, str2):
    cache = dict()
    def _ed(i, j):
        """Recursive helper, using cache."""
        if (i,j) in cache: 
            return cache[(i,j)]
        
        # Base cases
        if i == 0:
            result = j
        elif j == 0:
            result = i
            
        # Main recursion
        else:
            # 1 if letters differ (substitution is free if the letters are the same)
            substitution = 0 if str1[i - 1] == str2[j - 1] else 1
            result = min([
                    _ed(i - 1, j) + 1,  # insertion
                    _ed(i, j - 1) + 1,  # deletion
                    _ed(i - 1, j - 1) + substitution  # substitution, free if letters match  
            ])
        cache[(i,j)] = result
        return result
    
    return _ed(len(str1), len(str2))

In [8]:
def levenshtein_explicit(str1, str2, verbose=False):
    prev_num_edits = range(len(str1) + 1)
    for j in range(1, len(str2) + 1):
        num_edits = [prev_num_edits[0] + 1]
        for i in range(1, len(str1) + 1):
            # 1 if letters differ (substitution is free if the letters are the same)
            substitution = 0 if str1[i - 1] == str2[j - 1] else 1
            result = min([num_edits[i - 1] + 1,
                          prev_num_edits[i] + 1,
                          prev_num_edits[i - 1] + substitution
            ])
            num_edits.append(result)
        if verbose:
            print(prev_num_edits)
        prev_num_edits = num_edits
    if verbose:
        print(prev_num_edits)
    return prev_num_edits[len(str1)]

In [14]:
# Substitution.
levenshtein_explicit('abc', 'dbc', verbose=True)

In [10]:
# Deletion.
levenshtein_explicit('abc', 'ac')

In [11]:
# Insertion.
levenshtein_explicit('ac', 'abc')

In [12]:
# All of the above.
levenshtein_cache('kitten', 'sitting')

In [13]:
# Fun!
levenshtein_cache('w266 class', 'with 6 classic tricks')